<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Los-Angeles-City-Civic-Data" data-toc-modified-id="Los-Angeles-City-Civic-Data-1">Los Angeles City Civic Data</a></span></li><li><span><a href="#Building-Permit-Data" data-toc-modified-id="Building-Permit-Data-2">Building Permit Data</a></span><ul class="toc-item"><li><span><a href="#Socrata-API-and-managing-the-rate-limits" data-toc-modified-id="Socrata-API-and-managing-the-rate-limits-2.1">Socrata API and managing the rate limits</a></span></li><li><span><a href="#Inspecting-and-Cleaning-the-Data" data-toc-modified-id="Inspecting-and-Cleaning-the-Data-2.2">Inspecting and Cleaning the Data</a></span><ul class="toc-item"><li><span><a href="#Checking-for-Duplicates" data-toc-modified-id="Checking-for-Duplicates-2.2.1">Checking for Duplicates</a></span></li><li><span><a href="#Census-Tract" data-toc-modified-id="Census-Tract-2.2.2">Census Tract</a></span></li><li><span><a href="#Assessor's-Parcel-Number" data-toc-modified-id="Assessor's-Parcel-Number-2.2.3">Assessor's Parcel Number</a></span></li><li><span><a href="#Geospatial-Data" data-toc-modified-id="Geospatial-Data-2.2.4">Geospatial Data</a></span></li></ul></li></ul></li><li><span><a href="#Census-Data:-Merge-on-census_tract" data-toc-modified-id="Census-Data:-Merge-on-census_tract-3">Census Data: Merge on census_tract</a></span></li><li><span><a href="#LA-County-Parcel-Data-and-Joining-on-APN" data-toc-modified-id="LA-County-Parcel-Data-and-Joining-on-APN-4">LA County Parcel Data and Joining on APN</a></span></li><li><span><a href="#Geospatial-Data-and-Joining-on-Geometries" data-toc-modified-id="Geospatial-Data-and-Joining-on-Geometries-5">Geospatial Data and Joining on Geometries</a></span></li><li><span><a href="#Putting-it-all-together-into-an-ETL-class" data-toc-modified-id="Putting-it-all-together-into-an-ETL-class-6">Putting it all together into an ETL class</a></span></li></ul></div>

# Los Angeles City Civic Data

Trying to perform any sort of analysis or modeling of LA City land use issues is not just a exercise in exploratory data analysis and model building, its an exercise in data discovery and extraction. When I first came across <a href='https://data.lacity.org'>https://data.lacity.org</a>, with its over 1,000 datasets I thought that I would have pretty much everything I needed in one location to model the probabilty that an ADU (accesory dwelling unit), once someone applies for a permit, would be completed (see my post "Predicting ADU Completion"). Not so. 

Taking my clues about which factors influence ADU development projects from a design and architecture <a href='https://www.kcrw.com/culture/shows/design-and-architecture/dodger-stadium-upgrades-an-adu-is-born/at-long-last-an-adu-is-born-in-highland-park'>segment</a> on my local public radio station KCRW I soon realized that I would need to source data about building permits, parcel sizes, median incomes, historic preservation overlay zones and hillside ordinances. All of these datasets are maintained by different government entities. Some of the municipal services that we enjoy are provisioned by the city and others by the county. They each have their own data hubs (LA county's is <a href='https://data.lacounty.gov'>data.lacounty.gov</a>). They both contain some demographic data from the US Census, but without knowing how frequently it is updated I thought it better to source that data from the Census Bureau itself.

While it was quite straight forward to download some datasets, I found the API's provided for others to be querky at best. And then when the time came to merge them  together I often found that the most logical common key shared by two datasets to be represented differently in each. This post is meant to be a roadmap for anyone wanting to take advantage of the trove of publicly available data for the city of Los Angeles in order to understand, and improve, what makes our city tick.

# Building Permit Data
Luckily I had found on the LA City data hub what looked like to be a curated dataset of ADU permits applied for and issued during the six months between July 1, 2017 and December 31, 2017. At the onset of this project I though I could create my own dataset of ADU projoects from the larger dataset of all LA city building permits, but I don't believe there was a single identifying code for ADU's in LA's permitting system. Also, the ADU may be part of a larger project and permitted as such. Thankfully some helpful soul in the LA City IT department identified six months of permits for ADU's, most likely for someone's study, and isolated them into their own public data set. The status of all the permits in this set are updated once a week since their issuance. 

## Socrata API and managing the rate limits
The ADU dataset that I'm going to work with contains geospatial data that will have to be manipulated later, although not a geospatial databaser per se. Many of the non-geospatial datasets I've run across on municipal sites use the Socrata API. There is quite a bit of documentation available, even regarding this <a href='https://dev.socrata.com/foundry/data.lacity.org/r9zn-9ttc'>dataset</a> specifically (scroll down to the bottom of the page and click on the "Python Pandas" tab). When I first downloaded this data set I found that the rate limiting employed would not allow me to extract the entire dataset in one go. Given that set is fairly small (7,778 records) I found the rate limiting to be rather sever.

In order to get around this issue, and pull the entire dataset directly into memory, I had to write a loop that extracted chunks of it at a time. This knowledge came from this very helpful <a href='http://holowczak.com/getting-started-with-nyc-opendata-and-the-socrata-api/5/'>post</a> (see "Paging Through Data") by <a href='http://holowczak.com/about-me/'>Rich Holowczak</a> who was having the same issue with New York City data. Good to know that I'm not alone.

When I returned to this data set almost a year later I found that I could pull the entire data set in one, clean query. Thank you again to whoever it was in the LA City IT department that improved this process.

If you don't already have the sodapy package installed, please do so now. It's an easy pip install.

In [1]:
# Import the necessary packages
import numpy as np
import pandas as pd
from sodapy import Socrata
import censusdata
import geopandas as gpd
import requests
import re

In [2]:
# Import your api_token created for the LA City Datahub, as well as your username and password
with open('./data/la_city_key', 'r') as f:
    api_token = f.readline().replace('\n','')
with open('./data/la_city_username', 'r') as f:
    usrn = f.readline().replace('\n','')
with open('./data/la_city_password', 'r') as f:
    psswrd = f.readline().replace('\n','')

# The url we're using is the below
url = "data.lacity.org"

In [3]:
# Instantiate the Socrata client
client = Socrata(url, api_token, username=usrn, password=psswrd)

In [4]:
# Define the dataset we're requesting. In this case it's the "ADU permits 7/1/17-12/31/17(JB2)" db whose unique
# identifier is r9zn-9ttc. Then determine how many records are in the data set using the Socrata client and the 
# SQL count query
adu_permits = "r9zn-9ttc"
record_count = client.get(adu_permits, select="COUNT(*)")
record_count

[{'count': '7778'}]

In [5]:
# There are 7,778 records in the set, so I'm setting the limit to 8000 in my first attemp to download the dataset
data_raw = client.get('r9zn-9ttc', limit=8000)

This query took less than 5 seconds to run.

In [6]:
# Make a pandas DataFrame out of the raw data
data_df = pd.DataFrame(data_raw)

As mentioned above, downloading the data a year ago required me to do so in chucks. I going to put the code I used then below just for reference, in the event I need to go back to that method of data extraction.

In [7]:
# client.timeout = 50
# start = 0
# chunk_size = 1000
# data = []
# while True:
#      data.extend(client.get(adu_permits, offset=start, limit=chunk_size))
#      start = start + chunk_size
#      if (start > int(record_count[0]['count']) ):
#         break
        
# data_df = pd.DataFrame(data)

In [8]:
#Setting the index to False so I don't save the index as a column
data_df.to_csv('./data/adu_records.csv', index=False, float_format=str)

I would save this file locally so that I didn't have to do this every time I want to work on it back when I had to extract it in small chunks.

In [9]:
# data_df = pd.read_csv('./data/adu_records.csv')

## Inspecting and Cleaning the Data

In [10]:
# The first step I take with any new data set is just to look down the list of all of the data types in the set
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7778 entries, 0 to 7777
Data columns (total 61 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   zip_code                                 7778 non-null   object
 1   address_end                              7778 non-null   object
 2   work_description                         7778 non-null   object
 3   :@computed_region_qz3q_ghft              6773 non-null   object
 4   :@computed_region_k96s_3jcv              6773 non-null   object
 5   reference_old_permit                     7778 non-null   object
 6   principal_first_name                     3856 non-null   object
 7   :@computed_region_kqwf_mjcx              6773 non-null   object
 8   census_tract                             7778 non-null   object
 9   permit_category                          7778 non-null   object
 10  latest_status                            7778 non-null   obj

The first, most obvious issue with this dataset is the number of columns - 61. The description of the <a href="https://data.lacity.org/City-Infrastructure-Service-Requests/ADU-permits-7-1-17-12-31-17-JB2-/r9zn-9ttc">dataset</a> on data.lacity.org indicates that there should be 56 columns. So by inspection we see that for some reason there are 6 additional columns that begin with an ":@" symbol. Also, the column "event" isn't included in the download that is listed on the site. Let's have a look at the "@ ..." columns

In [11]:
data_df[[':@computed_region_qz3q_ghft',
         ':@computed_region_k96s_3jcv',
         ':@computed_region_kqwf_mjcx',
         ':@computed_region_2dna_qi2s',
         ':@computed_region_tatf_ua23',
         ':@computed_region_ur2y_g4cx']].head()

,:@computed_region_qz3q_ghft,:@computed_region_k96s_3jcv,:@computed_region_kqwf_mjcx,:@computed_region_2dna_qi2s,:@computed_region_tatf_ua23,:@computed_region_ur2y_g4cx
0,23448,493,11,1,619,NaN
1,23448,493,11,1,619,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,23673,371,9,93,598,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


These columns do not look like a recognizable value, and since they are not part of the data descriptions on the LA City Datahub site, I'm going to remove them.

In [12]:
del data_df[':@computed_region_qz3q_ghft']
del data_df[':@computed_region_k96s_3jcv']
del data_df[':@computed_region_kqwf_mjcx']
del data_df[':@computed_region_2dna_qi2s']
del data_df[':@computed_region_tatf_ua23']
del data_df[':@computed_region_ur2y_g4cx']

### Checking for Duplicates
One of the first item I check for is duplicate records. I'll do this here with the permit number as specified by the <a href='https://dev.socrata.com/foundry/data.lacity.org/r9zn-9ttc'>data dictionary</a> of this set.

In [13]:
data_df['pcis_permit'].duplicated().value_counts()

False    7778
Name: pcis_permit, dtype: int64

So I appears that there are no duplicates in the set. Great!

### Census Tract
The census tract number is one of the more granular levels of census data, much more so that zip codes for the most part. When looking at a single city, performing analysis at the census tract level provides much more nuance than the zip code level. Often in the city of Los Angeles a Neighborhood Council, the lowest level of city government, contains more than one census tract. I have seen data available for and even smaller footprint, the block level, but not as consistently as census tract.

Given the importance of census tract level data, let's examine the 'census tract' field in the ADU data set.

In [14]:
data_df['census_tract'].head()

0    1998.00
1    1998.00
2    1873.00
3    1836.20
4    1082.02
Name: census_tract, dtype: object

Census tracks can be expressed in a format that is XXXX.XX where the last two digits take forms such as: 01, 02, 10, 20, etc. I've also seen census tracts written as a six character strings without a period. The first 5 values look good, but let's check the data type for each record, just to be sure.

In [15]:
data_df['census_tract'].apply(lambda x: type(x)).value_counts()

<class 'str'>    7778
Name: census_tract, dtype: int64

All value are strings, so let's remove the "." and create a 6 character value for each record. This will make it easier to join to census data later.

In [16]:
# Revome the "." with a replace method
data_df['census_tract'] = data_df['census_tract'].apply(lambda x: x.replace('.', ''))

Let's check that all of the records have the same 6 character length.

In [17]:
data_df['census_tract'].apply(lambda x: len(x)).value_counts()

6    7778
Name: census_tract, dtype: int64

Looks good - all 7,778 records of 'census_tract' are 6 characters long.

### Assessor's Parcel Number
The Assessor's Parcel Number, or APN, is another key value that we want to pay special attention to it since is uniquely identifies individual properties. If you've ever paid a property tax bill with a paper check in Los Angeles, you've written the APN number on the return envelop provided. It's a 9 digit number in the format: XXXX-XX-XXX. The first four digits are the assessor's book number, the next two are the assessor's page, and the last three designate the parcel.

Inspecting the columns in the ADU dataset we find that the APN number has been recorded in three separate fields, one each for the book, page and parcel. Let's put it back together, with the dashes, so that we have a column that has the full APN number. This will make it easier to merge dataset from other sources, such as LA County, that use the APN as a unique identifier.

First let's examine the values in each of the columns to educate ourselves on the format of the data.

In [18]:
data_df[['assessor_book','assessor_page','assessor_parcel']].head()

,assessor_book,assessor_page,assessor_parcel
0,5211,002,022
1,5211,002,022
2,5438,013,001
3,5484,027,012
4,2701,001,040


But before we convert the data set to type str, let's check the data types of each value, just like we did with the census_tract.

In [19]:
print(data_df['assessor_book'].apply(lambda x: type(x)).value_counts())
print(data_df['assessor_page'].apply(lambda x: type(x)).value_counts())
print(data_df['assessor_parcel'].apply(lambda x: type(x)).value_counts())

<class 'str'>    7778
Name: assessor_book, dtype: int64
<class 'str'>    7778
Name: assessor_page, dtype: int64
<class 'str'>    7778
Name: assessor_parcel, dtype: int64


This all looks good, so we'll go ahead and create one record for APN by concatenating each of the individual fields.

In [20]:
data_df['APN'] = data_df['assessor_book']+'-'+data_df['assessor_page']+'-'+data_df['assessor_parcel']
del data_df['assessor_book']
del data_df['assessor_page']
del data_df['assessor_parcel']

### Geospatial Data

I have found most municipal datasets to be either a be a proper geospatial data set (meaning is stored in a common geospatial file format) or not. When working with geospatial data I would prefer to work with geoJSON's. The JSON format is a staple of the online world and can be used in simple NoSQL databases. Thus, if available I will ususlly download the geoJSON version of the data set. 

Inspecting the ADU data set we see a column title location, which contains latitude and longitude data, presumably of the parcel where the ADU is to be constructed.

Let's take a look at this record since we will need it to merge any data that is stored by defined geographic regions. 

In [21]:
data_df['location_1'].head()

0    {'latitude': '34.06971', 'human_address': '{"a...
1    {'latitude': '34.06971', 'human_address': '{"a...
2                                                  NaN
3    {'latitude': '34.11747', 'human_address': '{"a...
4                                                  NaN
Name: location_1, dtype: object

It looks like a dictionary with lattitude and longitude as a couple of the key, value paris. I'm going to take a look at just one record so I can see the format of it more closely.

In [22]:
data_df['location_1'].iloc[1]

{'latitude': '34.06971',
 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}',
 'needs_recoding': False,
 'longitude': '-118.20629'}

Yes indeed. It's relatively easy to extract, but this is what I meant earlier when I stated that the ADU data set has geospatial data in it, but it isn't a geospatial database. We'll need to extract the latitude and longitude into their own columns.

In [23]:
data_df['latitude'] = data_df['location_1'].apply(lambda x: x['latitude'] if isinstance(x, dict) else None)
data_df['latitude'] = data_df['latitude'].astype(float)
data_df['latitude'].notnull().sum()

6773

In [24]:
data_df['longitude'] = data_df['location_1'].apply(lambda x: x['longitude'] if isinstance(x, dict) else None)
data_df['longitude'] = data_df['longitude'].astype(float)
data_df['longitude'].notnull().sum()

6773

So now we have two columns for geospatial records: latitude and longitude. For now we'll just leave them as type float, and later we will use GeoPandas to convert them into the geo type point. This will enable us to use GeoPandas to determine if a location resides in a certain geographic areas with specific characteristics, and then merge those characteristics into our ADU dataset so that we can use them for modeling.

We can remove the location_1 column now.

In [25]:
del data_df['location_1']

# Census Data: Merge on census_tract

First let's download the data. I'd like to use median household income data in my model. The <a href="https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf"> Census Data API User Guide</a> is a nice document that explains how their API works, but before writing any Python code I first  use their <a href="https://data.census.gov/cedsci/advanced">GUI</a> query tool to get a feel of what is available. The API User Guide refers you to a their <a href="https://www.census.gov/data/developers/updates/new-discovery-tool.html">Discory Tool</a>, which, unless I'm missing something, is basically a very long list of data sets in various formats. I use the <a href="https://api.census.gov/data.html">html verion</a>. After some research I learned that I want to use the American Community Survey - 5 year data for median income. I go to that table in the Discovery Tool and start looking through the <a href="https://api.census.gov/data/2009/acs/acs5/variables.html">variables</a>. Oh my, a table that is even longer than the previous one. So long that it takes Chrome serval seconds to do a key word search of the page. So I start using command-F to search for the terms "income", "median income", "household income". I finally find what looks to be the correct variable name - B19013.

Another way to do this is to use an API wrapper. There are a few out there for US Census data. In fact, at the bottom of the API User Guide they refer you to this <a href="https://pypi.org/project/census/0.5/">one</a>. I however used <a href="https://pypi.org/project/CensusData/">censusdata</a>. I think it is most useful in searching through the census tables. As you will see below, you can query specific words and phrases and it generating a list of tables and names. This was how I locate B19013_001E as the table for household median income.

I make a few spot checks against the GUI for specific tracks - they matched! So I am confident that this is the data I should be using.

When it comes time to download a dataset of median income for all census tracts in Los Angeles, I decide to use the requests library and not a wrapper. So following the API User Guide I construct the correct URL. I find it helpful to first play around with the syntax in my browser. It is just a quick way to fiddle the url so until it is correct before pulling the data into my notebook with requests.

In [26]:
# I'm not going to run this in this notebook since the output is quite long
# censusdata.search('acs5', 2018, 'label', 'household income')

In [27]:
# An example of information on educational attainment
censusdata.printtable(censusdata.censustable('acs5', '2018', 'B23025'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B23025_001E  | EMPLOYMENT STATUS FOR THE POPU | !! Estimate Total                                        | int  
B23025_002E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total In labor force                      | int  
B23025_003E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Civilian labor fo | int  
B23025_004E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_005E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_006E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Armed Forces      | int  
B23025_007E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total Not in labor force      

In [28]:
# An example of information on employment
censusdata.printtable(censusdata.censustable('acs5', '2018', 'B23025'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B23025_001E  | EMPLOYMENT STATUS FOR THE POPU | !! Estimate Total                                        | int  
B23025_002E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total In labor force                      | int  
B23025_003E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Civilian labor fo | int  
B23025_004E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_005E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_006E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Armed Forces      | int  
B23025_007E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total Not in labor force      

The amount of data and the number of cross tabulations of that data the the Census Bureau provides is truly incredible. I'm a huge fan. But, getting back to the task at hand, I locate the table for median household income for which I was searching.

In [29]:
censusdata.printtable(censusdata.censustable('acs5', 2018, 'B19013'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19013_001E  | MEDIAN HOUSEHOLD INCOME IN THE | !! Estimate Median household income in the past 12 month | int  
-------------------------------------------------------------------------------------------------------------------


As a note, in the API documentation the NAME refers to the rather cryptic looking variable codes - B19013, etc. It took me a bit to catch on to this. I finally found that this was the golden ticket: https://api.census.gov/data/2019/acs/acs5?get=B19013_001E&for=tract:*&in=state:06&in=county:037

And so used it in the requests module to download the data into my notebook.

In [30]:
url='https://api.census.gov/data/2019/acs/acs5?get=B19013_001E&for=tract:*&in=state:06&in=county:037'
median_income_raw = requests.get(url)

In [31]:
median_income_raw

<Response [200]>

Response 200, or in plain English, I got something

In [32]:
# Converting it into a json for readability
median_income_raw.json()[:10]

[['B19013_001E', 'state', 'county', 'tract'],
 ['82917', '06', '037', '482702'],
 ['114831', '06', '037', '500201'],
 ['133125', '06', '037', '500202'],
 ['102875', '06', '037', '500300'],
 ['53500', '06', '037', '500500'],
 ['55446', '06', '037', '500900'],
 ['43929', '06', '037', '501400'],
 ['102292', '06', '037', '501501'],
 ['68783', '06', '037', '501802']]

It's a list of lists. The first row is the column headers, so make a DataFrame with all rows after the first and use the first row and the columns in the dataframe constructor.

In [33]:
median_income_df = pd.DataFrame(data=median_income_raw.json()[1:], columns=median_income_raw.json()[:1][0])

In [34]:
median_income_df.head()

,B19013_001E,state,county,tract
0,82917,06,037,482702
1,114831,06,037,500201
2,133125,06,037,500202
3,102875,06,037,500300
4,53500,06,037,500500


In [35]:
median_income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346 entries, 0 to 2345
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   B19013_001E  2346 non-null   object
 1   state        2346 non-null   object
 2   county       2346 non-null   object
 3   tract        2346 non-null   object
dtypes: object(4)
memory usage: 73.4+ KB


In [36]:
median_income_df.rename(columns={'B19013_001E':'median_income','tract':'census_tract'}, inplace=True)
del median_income_df['state']
del median_income_df['county']

All the ADU's are in Los Angeles city, and the median income data is for every census tract in Los Angeles county,
so by doing a left join of the median income data onto the ADU dataset with the census_tract as the key, I would expect most all of the records to have an associated median income after the merge. Note every record of the 7,778 ADU dataset has a value for census_tract - see .info() at the beginning.

In [37]:
#Let's perform the merge and see what we get. 
pd.merge(data_df, median_income_df, on='census_tract', how='left', indicator=True).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7778 entries, 0 to 7777
Data columns (total 56 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   zip_code                                 7778 non-null   object  
 1   address_end                              7778 non-null   object  
 2   work_description                         7778 non-null   object  
 3   reference_old_permit                     7778 non-null   object  
 4   principal_first_name                     3856 non-null   object  
 5   census_tract                             7778 non-null   object  
 6   permit_category                          7778 non-null   object  
 7   latest_status                            7778 non-null   object  
 8   initiating_office                        7778 non-null   object  
 9   applicant_first_name                     7639 non-null   object  
 10  zone                                

This looks quite good. Only 20 of the records in the ADU data set did not have a match on census_tract with the median income data from the census bureau. (Total number of records = 7778, number of median_income values = 7758.) So the correction we applied when the cleaned the ADU data set seems to have worked. 

I have, however, encountered situations where the match is not so good. To examine those cases in more detail I would first perform the same merge, only as an outer join, and then take a look at the individual records that did NOT match in both data sets, since you don't know at the onset where an error may lie.

Let's do that here just for practice. I like to use the "indicator" flag so that I can monitor the merge. It will produce a column "_merge" that indicated where the value for the key came from for each record, either the left table, the right table or both.

In [38]:
test = pd.merge(data_df, median_income_df, on='census_tract', how='outer', indicator=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9319 entries, 0 to 9318
Data columns (total 56 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   zip_code                                 7778 non-null   object  
 1   address_end                              7778 non-null   object  
 2   work_description                         7778 non-null   object  
 3   reference_old_permit                     7778 non-null   object  
 4   principal_first_name                     3856 non-null   object  
 5   census_tract                             9319 non-null   object  
 6   permit_category                          7778 non-null   object  
 7   latest_status                            7778 non-null   object  
 8   initiating_office                        7778 non-null   object  
 9   applicant_first_name                     7639 non-null   object  
 10  zone                                

So what we have here is a merged dataset with 9319 records. In order to understand the output I keep the chart below handy as a nemonic to remember the basic permutations that are performed with outer joins. Here you see that the 9319 records are made up of:
1. Some number of records that matched on census_tract, so the records from the ADU dataset and those from the median income dataset became one
2. Some number of ADU records that had a census_tract that wasn't in the median income dataset, and 
3. Some number of median income records that had a census_tract that wasn't in the ADU data set.


[diagram]



 

In [39]:
# Calculate the number of median income records that didn't find a match of census_tract in the ADU records
test[test['_merge']=='right_only'].count()

zip_code                                      0
address_end                                   0
work_description                              0
reference_old_permit                          0
principal_first_name                          0
census_tract                               1541
permit_category                               0
latest_status                                 0
initiating_office                             0
applicant_first_name                          0
zone                                          0
contractor_state                              0
license_expiration_date                       0
principal_middle_name                         0
license_type                                  0
valuation                                     0
pcis_permit                                   0
applicant_relationship                        0
contractor_city                               0
address_start                                 0
street_name                             

In [40]:
# Calculate the number of ADU records that didn't find a match of census_tract in the median income records
test[test['_merge']=='left_only'].count()

zip_code                                   20
address_end                                20
work_description                           20
reference_old_permit                       20
principal_first_name                       10
census_tract                               20
permit_category                            20
latest_status                              20
initiating_office                          20
applicant_first_name                       20
zone                                       18
contractor_state                            5
license_expiration_date                     7
principal_middle_name                       6
license_type                               20
valuation                                  20
pcis_permit                                20
applicant_relationship                     19
contractor_city                             5
address_start                              20
street_name                                20
street_suffix                     

In [41]:
# Calculate the number of records that matched on census_tract
test[test['_merge']=='both'].count()

zip_code                                   7758
address_end                                7758
work_description                           7758
reference_old_permit                       7758
principal_first_name                       3846
census_tract                               7758
permit_category                            7758
latest_status                              7758
initiating_office                          7758
applicant_first_name                       7619
zone                                       7754
contractor_state                           3603
license_expiration_date                    4742
principal_middle_name                      2405
license_type                               7758
valuation                                  7758
pcis_permit                                7758
applicant_relationship                     7632
contractor_city                            3604
address_start                              7758
street_name                             

The sum of these three calculation should equal the maximum total number of records obtain from the outer merge, and in fact, they do!!

In [42]:
20 + 1541 + 7758

9319

So now let's take a deep look at the 20 unmatched records from the ADU (left) data set that didn't find a match with the median income data set.

In [43]:
# Examine the 20 unmatched records
test[test['_merge']=='left_only']['census_tract'].value_counts()

930401    14
192400     2
195900     1
269600     1
219900     1
120000     1
Name: census_tract, dtype: int64

So there are only 6 unique census_tracts that don't have a matching value in the median income data set. Let's take a look at each one of them.

In [44]:
test[test['census_tract']=='930401'][['address_start','street_direction', 'street_name','street_suffix', 'suffix_direction']]

,address_start,street_direction,street_name,street_suffix,suffix_direction
1756,23009,W,BURBANK,BLVD,NaN
1757,23063,W,CANZONET,ST,NaN
1758,5235,N,WOODLAKE,AVE,NaN
1759,23236,W,CANZONET,ST,NaN
1760,23211,W,MARIANO,ST,NaN
1761,22952,W,OSTRONIC,DR,NaN
1762,22946,W,OSTRONIC,DR,NaN
1763,22923,W,OSTRONIC,DR,NaN
1764,23035,W,LEONORA,DR,NaN
1765,23016,W,LEONORA,DR,NaN


Using Google Maps and the US Census Bureau's GUI query tool I find this is mis-coded in the ADU data. It should be 137000. All of the streets above are in that tract.

In [45]:
test[test['census_tract']=='120000'][['address_start','street_direction', 'street_name','street_suffix', 'suffix_direction']]

,address_start,street_direction,street_name,street_suffix,suffix_direction
7626,15126,W,BURTON,ST,NaN


There are various combinations of Burton St in Los Angeles. By looking at Google Maps and the US Census Bureau's GUI query tool, this street address is only valid without the "W" prefix but the 120001 census tract does have a Burton St, just not one with this street number. So it is difficult to determine where the error lies, in the census_tract or the address. Did someone use the wrong address when they looked up the census_tract, when it should be totally different, or did they look up the street address wrong when they recorded it, and then got the census_tract wrong by one digit. Given that we cannot really be sure, I'm going to leave these records as unmatched.

In [46]:
test[test['census_tract']=='192400'][['address_start','street_direction', 'street_name','street_suffix', 'suffix_direction']]

,address_start,street_direction,street_name,street_suffix,suffix_direction
6257,600,N,PLYMOUTH,BLVD,NaN
6258,600,N,PLYMOUTH,BLVD,NaN


Again using Google Maps and the US Census Bureau's GUI query tool I find that this is mis-coded in the ADU data. It should be 192410. This address is in that tract.

Following the same process for the remaining 3 census_tract values I find that they are simply incorrect in the ADU data set. 

The correction are as follows:
- 219900 should be 219902
- 195900 should be 195903
- 269200 should be 269601

In [47]:
# Correct census_tract errors in the ADU data set
data_df['census_tract'] = data_df['census_tract'].apply(lambda x: '137000' if x=='930401' else x)
data_df['census_tract'] = data_df['census_tract'].apply(lambda x: '192410' if x=='192400' else x)
data_df['census_tract'] = data_df['census_tract'].apply(lambda x: '219902' if x=='219900' else x)
data_df['census_tract'] = data_df['census_tract'].apply(lambda x: '195903' if x=='195900' else x)
data_df['census_tract'] = data_df['census_tract'].apply(lambda x: '269201' if x=='269600' else x)

So we can now add the median income data to the DataFrame we want to use for modeling.

In [48]:
data_df = pd.merge(data_df, median_income_df, on='census_tract', how='left')

It is often said that 80% of the analyst's job is cleaning data, and this is another example of it.

# LA County Parcel Data and Joining on APN
One of the features I would like to add to my ADU predictive model is lot size. My intuition is that it is easier to build on a larger lot due to having more space on which to maneuver. For this I've gone to the LA County Geohub where <a href="https://geohub.lacity.org/datasets/6d85cb5f5f5641c6aa95203849ca05bb_0?geometry=-128.604%2C32.202%2C-107.994%2C35.396">detailed parcel records</a> can be found. This data set has over 2.4 million records with ninety-one columns. 

As I mentioned earlier in this post, sometimes the same data is hosted on both the county and the city datahubs. This is the case with parcel data. The LA City datahub has two data sets of parcels, <a href="https://geohub.lacity.org/datasets/lacounty::parcels">one with all 2.4 million records</a>, and only the ability to download the file, presumably due to its size, and <a href="https://geohub.lacity.org/datasets/la-city-parcels?geometry=-120.986%2C33.625%2C-115.833%2C34.422">another</a> with just the parcels in LA City. This reduces the record count to 882K, but for some reason this data set has fewer columns. LA City provides an an API to query this data set directly. Unfortunately the one column I was hoping to get, the size of the lot, isn't present. Overall I'm finding that the LA City data hub does a better job of documentation, and given the sometimes unintelligible naming of columns, this can be a great asset. Given that it seems as if LA City has reproduced the complete parcel data set from the county, I choose to source the data from its owner - LA County.

In [49]:
# parcels_raw = gpd.read_file('https://opendata.arcgis.com/datasets/6d85cb5f5f5641c6aa95203849ca05bb_0.geojson')

(Note, this is a geojson dataset, so a proper geospatial db, and thus I can query with GeoPandas directly.)

I first tried to query the entire 2.4 million records, but after 10 minutes of waiting I halted that exercise.

I then used the "API Explorer" available on their site to generate a url that will query a reduced number of columns. I change my query to request only seven columns, an object ID, the 5 values of square footage, and the lot size. This should make the request much smaller and indeed it does finish in seconds.

In [50]:
parcels_raw = requests.get('https://public.gis.lacounty.gov/public/rest/services/LACounty_Cache/LACounty_Parcel/MapServer/0/query?where=1%3D1&outFields=OBJECTID,APN,SQFTmain1,SQFTmain2,SQFTmain3,SQFTmain4,SQFTmain5,Shape.STArea()&outSR=4326&f=json')

In [51]:
parcels_raw

<Response [200]>

(Note that the API explorer generates a json file, not a geojson file, probably because the user does not have to include the "geometry" column. Thus I do not use GeoPandas but the requests library.)

But as I started using it I found that it only downloads 1000 records - just like the Socrata API did with the ADU data.

In [52]:
range(len(parcels_raw.json()['features']))

range(0, 1000)

So its seems that LA City foresaw this problem and thus only allows users to access the data via download and not an API. I decide the best thing to do is download the entire data set to my laptop and load it locally.

In [53]:
parcels_df = pd.read_csv('./data/LA_County_Parcel_Map_Service.csv')
parcels_df.info()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,18,28,35,42,49,74,75,76) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410913 entries, 0 to 2410912
Data columns (total 91 columns):
 #   Column                Dtype  
---  ------                -----  
 0   OBJECTID              int64  
 1   AIN                   object 
 2   APN                   object 
 3   SitusHouseNo          float64
 4   SitusFraction         object 
 5   SitusDirection        object 
 6   SitusUnit             object 
 7   SitusStreet           object 
 8   SitusAddress          object 
 9   SitusCity             object 
 10  SitusZIP              object 
 11  SitusFullAddress      object 
 12  TaxRateArea           float64
 13  TaxRateCity           object 
 14  AgencyClassNo         float64
 15  AgencyName            object 
 16  AgencyType            object 
 17  UseCode               object 
 18  UseCode_2             object 
 19  UseType               object 
 20  UseDescription        object 
 21  DesignType1           object 
 22  YearBuilt1            float64
 23  Effecti

Since I'm only going to the use lot size and the total square footage of all the buildings on the lot, I inspect just those columns.

In [54]:
parcels_df[['APN','SQFTmain1','SQFTmain2','SQFTmain3','SQFTmain4','SQFTmain5','Shape.STArea()']].head()

,APN,SQFTmain1,SQFTmain2,SQFTmain3,SQFTmain4,SQFTmain5,Shape.STArea()
0,7102-032-097,1122.0,NaN,NaN,NaN,NaN,263890.580078
1,7102-032-107,1361.0,NaN,NaN,NaN,NaN,263890.580078
2,7103-002-001,NaN,NaN,NaN,NaN,NaN,4858.358398
3,7103-002-003,1881.0,NaN,NaN,NaN,NaN,7503.089844
4,7103-003-005,7500.0,NaN,NaN,NaN,NaN,7482.393555


Since most lots won't have more than one building, and the dataset is carrying null values in those cases, I decide to impute those records with zeros so that I can sum the square footage from every building into one number for my model.

In [55]:
parcels_df['SQFTmainTot']=parcels_df['SQFTmain1'].fillna(0)+parcels_df['SQFTmain2'].fillna(0)+\
    parcels_df['SQFTmain3'].fillna(0)+parcels_df['SQFTmain4'].fillna(0)+parcels_df['SQFTmain5'].fillna(0)

And then save it locally so that I don't have to work from the same monster dataset if I re-run the code.

In [56]:
parcels_sub = parcels_df[['APN','SQFTmainTot', 'Shape.STArea()']]
parcels_sub.to_csv('parcels_sub.csv')
parcels_sub.head()

,APN,SQFTmainTot,Shape.STArea()
0,7102-032-097,1122.0,263890.580078
1,7102-032-107,1361.0,263890.580078
2,7103-002-001,0.0,4858.358398
3,7103-002-003,1881.0,7503.089844
4,7103-003-005,7500.0,7482.393555


So with this clean dataset, let's do a test merge to our ADU data set with median income. I would expect virtually all the records to have a match since the APN's in the ADU data set are all from LA City, and the parcels data should be all parcel in LA County.

In [57]:
test = pd.merge(data_df, parcels_sub, on='APN', how='left', indicator=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7778 entries, 0 to 7777
Data columns (total 58 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   zip_code                                 7778 non-null   object  
 1   address_end                              7778 non-null   object  
 2   work_description                         7778 non-null   object  
 3   reference_old_permit                     7778 non-null   object  
 4   principal_first_name                     3856 non-null   object  
 5   census_tract                             7778 non-null   object  
 6   permit_category                          7778 non-null   object  
 7   latest_status                            7778 non-null   object  
 8   initiating_office                        7778 non-null   object  
 9   applicant_first_name                     7639 non-null   object  
 10  zone                                

So there are 7778 - 7450 = 328 records in the ADU dataset that don't have a match in the parcel dataset. Not terrible but peculiar. About half of them are unique so it looks like there could be multiple permits for the same lot, or simply input errors with the APN.
 
Let's take a look at those records in the ADU data set whose APN did not match to any APN in the parcels data set.

In [58]:
test['_merge'].value_counts()

both          7450
left_only      328
right_only       0
Name: _merge, dtype: int64

In [59]:
len(test[test['_merge']=='left_only']['APN'].unique())

169

Interesting that only about half the records are unique. This seems a little funny. I'm going to do some spot checking to see if anything else jumps out.

In [60]:
# I chose tail just to mix it up.
test[test['_merge']=='left_only']['APN'].tail(10)

7671    2527-011-031
7678    5569-037-001
7680    2355-018-026
7709    5485-006-002
7716    5442-002-919
7746    4409-005-023
7747    4369-001-014
7750    4494-022-030
7752    4409-005-021
7763    4369-001-014
Name: APN, dtype: object

And now pick one randomly and try to match it to a record in the parcels data set by address instead of APN.

In [61]:
# query this one record for its street address
test[test['APN']=='4409-005-023'][['address_start', 'street_name']]

,address_start,street_name
7746,1105,RIVAS CANYON


In [62]:
# Now check the parcels data set for addresses that have these elements using a regex
for x in parcels_df['SitusAddress']:
    if re.search('1105 RIVAS CANYON', str(x)):
        print(x)

1105 RIVAS CANYON ROAD


In [63]:
# Now take the full street address and filter the parcels data set for its APN
parcels_df[parcels_df['SitusAddress']=='1105 RIVAS CANYON ROAD']

,OBJECTID,AIN,APN,SitusHouseNo,SitusFraction,SitusDirection,SitusUnit,SitusStreet,SitusAddress,SitusCity,...,LegalDescLineLast,LegalDescription,CENTER_LAT,CENTER_LON,CENTER_X,CENTER_Y,LAT_LON,Shape.STArea(),Shape.STLength(),SQFTmainTot
1571134,1571135,4409005030,4409-005-030,1105.0,,,,RIVAS CANYON ROAD,1105 RIVAS CANYON ROAD,PACIFIC PALISADES,...,DESC SEE ASSESSOR'S MAPS POR OF LOT 40,OFFICIAL MAP OF LOS ANGELES COUNTY FOR DESC SE...,34.04981,-118.516282,6.405275e+06,1.840911e+06,"34.049810, -118.516282",83450.136719,1215.682444,10000.0


And here we see that the APN for this address in the county data set is 4409-005-030, while its 4409-005-023 in the LA City ADU data set.

After having done this a few times it seems that the APN is incorrect in either the ADU data set or the parcels data set.

At this point I'm going to leave the 328 records unmatched. If I thought that the lot size feature would be a strong contributor to the predictive accuracy of the model I want to build, or if I find later that it is, I would add address as a match criteria. At this point I going to go with it as is since 328 records is only about 4% of the ADU records.

Let's go ahead and merge the parcel data on the the ADU data.

In [64]:
data_df = pd.merge(data_df, parcels_sub, on='APN', how='left')

And then clean up some of the naming.

In [65]:
data_df.rename(columns={'SQFTmainTot':'building_size', 'Shape.STArea()':'lot_size'}, inplace=True)

And then I'm going to create a column of the land size minus the building size to represent just the open space on the lot.

In [66]:
data_df['open_land']=data_df['lot_size']-data_df['building_size']

# Geospatial Data and Joining on Geometries
The third type of data we want to add is an indicator if the lot was on a hillside. The radio story mentioned earlier in this post suggested that hillside can be quite difficult to build on. I found an interesting <a href="https://buildcover.com/updates/las-proposed-hillside-adu-ban">blog post</a> from a company that manufactures pre-fabricated ADU's which was also warning of new hillside construction ordinances going into effect in Los Angeles.

In the LA City data hub a <a href="https://geohub.lacity.org/datasets/hillside-ordinance">geospatial dataset</a> of these areas already existed. We can use this to tag every record in the ADU data set as being on a hillside or not. This represents another way of merging data together - a spatial join. For this we are going to use the GeoPandas library to determine if the location of the project lies within the boundaries of a hillside. This type of spatial join is referred to as a point in polygon. 

So let's first download the hillside dataset. Since its a geospatial database, we can simply read the geojson file into a GeoDataFrame

In [67]:
hills_gdf = gpd.read_file('https://opendata.arcgis.com/datasets/3ac07567df1c4f3b916ac258e426e3f5_6.geojson')
hills_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   H_TYPE    344 non-null    object  
 1   OBJECTID  344 non-null    int64   
 2   TOOLTIP   344 non-null    object  
 3   geometry  344 non-null    geometry
dtypes: geometry(1), int64(1), object(2)
memory usage: 10.9+ KB


Ah, a mere 344 records, so this loaded like a breeze.

Before we merge this data set to the ADU data set we have to first convert the longitude and latitude columns in the ADU data set into a proper geospatial element, and define the projection. The projection is the crs, or coordinate reference system, that is used to project a round shape like the planet onto a flat surface that is a map. We'll use the crs of the hillside data set, since they must match to properly merge on the geometry features.

In [68]:
# convert the ADU data set to a GeoDataFrame with GeoPandas and create a point geometry from the lat-long columns
data_gdf = gpd.GeoDataFrame(data_df, geometry=gpd.points_from_xy(data_df.longitude, data_df.latitude))

In [69]:
# Assign it the same crs as the hillside data set
data_gdf.crs = hills_gdf.crs

In [70]:
# Delete some unnessary columns
del hills_gdf['OBJECTID']
del hills_gdf['TOOLTIP']

In [71]:
# With the spatial joing method of GeoPandas merge the hills_gdf GeoDataFrame onto the data_gdf GeoDataFrame
data_gdf = gpd.sjoin(data_gdf, hills_gdf, how='left', op="within")
data_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7778 entries, 0 to 7777
Data columns (total 61 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   zip_code                                 7778 non-null   object  
 1   address_end                              7778 non-null   object  
 2   work_description                         7778 non-null   object  
 3   reference_old_permit                     7778 non-null   object  
 4   principal_first_name                     3856 non-null   object  
 5   census_tract                             7778 non-null   object  
 6   permit_category                          7778 non-null   object  
 7   latest_status                            7778 non-null   object  
 8   initiating_office                        7778 non-null   object  
 9   applicant_first_name                     7639 non-null   object  
 10  zone                        

So it looks like 1694 records in our ADU data set are on a hillside

In [72]:
del data_gdf['index_right']

In [73]:
# Rename the hillside tag column to something more readable
data_gdf.rename(columns={'H_TYPE':'hillside'}, inplace=True)

In [74]:
# And now let's clean up the actual tag values so they are simply '1' is the record is of a hillside location
data_gdf['hillside']=data_gdf['hillside'].apply(lambda x: int(1) if isinstance(x,str) else np.nan)

As it turns out merging the geospatial data into the ADU data set was the easier than the key word merges. Of course I'm not doing the say validation in this merge as I did with the others. One could geo-locate the address of every ADU record, create a point with a buffer around, and then check to see if the lat-long data provided in the ADU dataset actually falls within the buffer. In a production setting where the validity of the locations is critical I would write a function that does this, but for the purpose of what I wanted to accomplish here, what we've done should be sufficient.

So we've gone through an example of assembling a data set of building permit data from the city of Los Angeles, parcel details from the county of Los Angeles, demographic data from the US Census and geospatial data from the city. We've also seen how to merge data set on a key, check the validity of the key itself, and how to perform a spatial join to include regional characteristics in a data set. This covers many of the situations you would need to build an interesting data set from multiple sources to address civic issues in the city of Los Angeles.

In [75]:
data_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7778 entries, 0 to 7777
Data columns (total 60 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   zip_code                                 7778 non-null   object  
 1   address_end                              7778 non-null   object  
 2   work_description                         7778 non-null   object  
 3   reference_old_permit                     7778 non-null   object  
 4   principal_first_name                     3856 non-null   object  
 5   census_tract                             7778 non-null   object  
 6   permit_category                          7778 non-null   object  
 7   latest_status                            7778 non-null   object  
 8   initiating_office                        7778 non-null   object  
 9   applicant_first_name                     7639 non-null   object  
 10  zone                        

# Putting it all together into an ETL class
And with that we've create a dataset on which to begin the modeling process. But, first lets pull everything we've done into a single function that can be imported into any other notebooks and use to create a working data set. I'm going to write the code so that it can pull the large parcel data from disk since we found that using the API to download even a fraction of it directly into memory wasn't feasible.

In [76]:
class ETL:
    def __init__(self, la_city_database, la_city_usrn, la_city_psswrd, la_city_token, limit=8000):
        self.la_city_database = la_city_database,
        self.la_city_usrn = la_city_usrn,
        self.la_city_psswrd = la_city_psswrd,
        self.la_city_token = la_city_token
        with open(la_city_token, 'r') as f:
            api_token = f.readline().replace('\n','')
        with open(la_city_usrn, 'r') as f:
            usrn = f.readline().replace('\n','')
        with open(la_city_psswrd, 'r') as f:
            psswrd = f.readline().replace('\n','')
        self.client = Socrata('data.lacity.org', api_token, username=usrn, password=psswrd)
    
    def __repr__(self):
        return f'Query for database {self.la_city_database[0]}'
    
    # Create getters and setters for class attributes
    @property
    def la_city_database(self):
        return self._la_city_database
    
    @la_city_database.setter
    def la_city_database(self, la_city_database):
        if not isinstance(la_city_database[0], str):
            raise ValueError('"la_city_database" must be a string.')
        self._la_city_database = la_city_database[0]
    
    @property
    def la_city_usrn(self):
        return self._la_city_usrn
    
    @la_city_database.setter
    def la_city_usrn(self, la_city_usrn):
        if not isinstance(la_city_usrn[0], str):
            raise ValueError('"la_city_usrn" must be the path to your LA City Datahub username in str format.')
        self._la_city_usrn = la_city_usrn[0]
    
    @property
    def la_city_psswrd(self):
        return self._la_city_psswrd
    
    @la_city_database.setter
    def la_city_psswrd(self, la_city_psswrd):
        if not isinstance(la_city_psswrd[0], str):
            raise ValueError('"la_city_psswrd" must be the path to your LA City Datahub password in str format.')
        self._la_city_psswrd = la_city_psswrd[0]
    
    @property
    def la_city_token(self):
        return self._la_city_token
    
    @la_city_token.setter
    def la_city_token(self, la_city_token):
        if not isinstance(la_city_token[0], str):
            raise ValueError('"la_city_token" must be a the path to your LA City Datahub token in str format.')
        self._la_city_token = la_city_token[0]
        
    def get_records(self):
        self.record_count = self.client.get(self.la_city_database, select="COUNT(*)")
        return self.record_count
        
    def get_data(self):
        self.data_raw = self.client.get(self.la_city_database, limit=8000)
        self.data_df = pd.DataFrame(self.data_raw)
        return self.data_df
        
    def clean_data(self, data):
        data_pdf = data.copy()
        
        # Delete columns starting with ':@'
        bad_cols = [x for x in data_pdf.columns if re.search(':@', str(x))]
        for col in bad_cols:
            del data_pdf[col]
        
        data_pdf['census_tract'] = data_pdf['census_tract'].apply(lambda x: x.replace('.', ''))
        
        # Create one APN column
        data_pdf['APN'] = data_pdf['assessor_book']+'-'+data_pdf['assessor_page']+'-'+data_pdf['assessor_parcel']
        del data_pdf['assessor_book']
        del data_pdf['assessor_page']
        del data_pdf['assessor_parcel']
        
        # Extract latitude and longitude into separate columns
        data_pdf['latitude'] = data_pdf['location_1'].apply(lambda x: x['latitude'] if isinstance(x, dict) else None)
        data_pdf['latitude'] = data_pdf['latitude'].astype(float)
        data_pdf['longitude'] = data_pdf['location_1'].apply(lambda x: x['longitude'] if isinstance(x, dict) else None)
        data_pdf['longitude'] = data_pdf['longitude'].astype(float)
        del data_pdf['location_1']
        
        # Fix errors in census_tract
        data_pdf['census_tract'] = data_pdf['census_tract'].apply(lambda x: '137000' if x=='930401' else x)
        data_pdf['census_tract'] = data_pdf['census_tract'].apply(lambda x: '192410' if x=='192400' else x)
        data_pdf['census_tract'] = data_pdf['census_tract'].apply(lambda x: '219902' if x=='219900' else x)
        data_pdf['census_tract'] = data_pdf['census_tract'].apply(lambda x: '195903' if x=='195900' else x)
        data_pdf['census_tract'] = data_pdf['census_tract'].apply(lambda x: '269601' if x=='269600' else x)
        
        return data_pdf
            
    def median_income(self, data):
        data_pdf = data.copy()
        
        url='https://api.census.gov/data/2019/acs/acs5?get=B19013_001E&for=tract:*&in=state:06&in=county:037'
        median_income_raw = requests.get(url)
        median_income_df = pd.DataFrame(data=median_income_raw.json()[1:], columns=median_income_raw.json()[:1][0])
        median_income_df.rename(columns={'B19013_001E':'median_income','tract':'census_tract'}, inplace=True)
        del median_income_df['state']
        del median_income_df['county']
        data_pdf = pd.merge(data_pdf, median_income_df, on='census_tract', how='left')
        
        return data_pdf
    
    def parcels(self, data, parcels_loc):
        data_pdf = data.copy()
        
        parcels_df = pd.read_csv(parcels_loc)
        parcels_df['SQFTmainTot']=parcels_df['SQFTmain1'].fillna(0)+parcels_df['SQFTmain2'].fillna(0)+\
            parcels_df['SQFTmain3'].fillna(0)+parcels_df['SQFTmain4'].fillna(0)+parcels_df['SQFTmain5'].fillna(0)
        parcels_sub = parcels_df[['APN','SQFTmainTot', 'Shape.STArea()']]
        data_pdf = pd.merge(data_pdf, parcels_sub, on='APN', how='left')
        data_pdf.rename(columns={'SQFTmainTot':'building_size', 'Shape.STArea()':'lot_size'}, inplace=True)
        data_pdf['open_land']=data_pdf['lot_size']-data_pdf['building_size']
        
        return data_pdf
    
    def hillsides(self, data):
        data_pdf = data.copy()
        
        hills_gdf = gpd.read_file('https://opendata.arcgis.com/datasets/3ac07567df1c4f3b916ac258e426e3f5_6.geojson')
        data_gdf = gpd.GeoDataFrame(data_pdf, geometry=gpd.points_from_xy(data_pdf.longitude, data_pdf.latitude))
        data_gdf.crs = hills_gdf.crs
        del hills_gdf['OBJECTID']
        del hills_gdf['TOOLTIP']
        data_gdf = gpd.sjoin(data_gdf, hills_gdf, how='left', op="within")
        del data_gdf['index_right']
        del hills_gdf['latitude']
        del hills_gdf['longitude']
        data_gdf.rename(columns={'H_TYPE':'hillside'}, inplace=True)
        data_gdf['hillside']=data_gdf['hillside'].apply(lambda x: int(1) if isinstance(x,str) else np.nan)
        
        return data_gdf